<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Подготовим-данные-для-машинного-обучения" data-toc-modified-id="Подготовим-данные-для-машинного-обучения-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Подготовим данные для машинного обучения</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

<b>Описание данных</b><br>

<b>Признаки</b><br>
<b>RowNumber</b> — индекс строки в данных<br>
<b>CustomerId</b> — уникальный идентификатор клиента<br>
<b>Surname</b> — фамилия<br>
<b>CreditScore</b> — кредитный рейтинг<br>
<b>Geography</b> — страна проживания<br>
<b>Gender</b> — пол<br>
<b>Age</b> — возраст<br>
<b>Tenure</b> — сколько лет человек является клиентом банка<br>
<b>Balance</b> — баланс на счёте<br>
<b>NumOfProducts</b> — количество продуктов банка, используемых клиентом<br>
<b>HasCrCard</b> — наличие кредитной карты<br>
<b>IsActiveMember</b> — активность клиента<br>
<b>EstimatedSalary</b> — предполагаемая зарплата<br><br>
<b>Целевой признак</b><br><br>
<b>Exited</b> — факт ухода клиента<br>

<b> План работы: </b>
1. Откроем файлы с данными, проведем предобработку данных.
2. Подготовим данные для машинного обучения: исследуем баланс классов и изучим модели без учёта дисбаланса
3. Учтём дисбаланс, применем несколько способов борьбы с дисбалансом
4. Проведем тестирование. Достигнем F1-меры не менее 0.59. Исследуем метрику AUC-ROC.

## Подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve, confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

In [2]:
data_clients = pd.read_csv('/datasets/Churn.csv')
data_clients.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [3]:
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


В столбце <b>"Tenure"</b> есть пропуски. Заменим их на минимальные значения.

In [4]:
data_clients['Tenure'] = data_clients['Tenure'].fillna(0)

In [5]:
data_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [6]:
data_clients.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.543400,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,3.111573,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,4.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Проверим на дубликаты

In [7]:
data_clients.duplicated().sum()

0

<b>Вывод</b><br>
Датасет содержить 10000 строк, без явных пропусков и дубликатов.
CustomerId — уникальный идентификатор клиента, так как дубликатов нет, его можно удалить.

## Исследование задачи

### Подготовим данные для машинного обучения

Удалим столбцы индентификаторы

In [8]:
data_clients_m = data_clients.drop(['RowNumber','CustomerId', 'Surname'], axis=1)
data_clients_m.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [9]:
data_clients_m['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [10]:
data_clients_m['Gender'].unique()

array(['Female', 'Male'], dtype=object)

Подготовим данные методом OHE, с защитой от дамми-ловушки

In [11]:
data_clients_m = pd.get_dummies(data_clients_m, drop_first=True)
data_clients_m.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


Применим модели: логистическая регрессия, случайный лес, решающее дерево. Разобъем данные - 60, 20, 20.

Разделим на признаки и целевой признак

In [12]:
features = data_clients_m.drop('Exited', axis=1)
target = data_clients_m['Exited']

features_train, features_validtest, target_train, target_validtest = train_test_split(features,
                                                    target,
                                                    train_size=0.6,
                                                    random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(features_validtest,
                                                    target_validtest,
                                                    train_size=0.5,
                                                    random_state=12345)
print(features_train.shape, features_valid.shape, features_test.shape)
print(target_train.shape, target_valid.shape, target_test.shape)

(6000, 11) (2000, 11) (2000, 11)
(6000,) (2000,) (2000,)


Решающее дерево.

In [13]:
model = DecisionTreeClassifier(random_state=12345, max_depth=6)
DTC_model = model.fit(features_train, target_train).score(features_valid, target_valid)
predicted_valid = model.predict(features_valid)
print("дерево решений", DTC_model)

дерево решений 0.858


Cлучайный лес.

In [14]:
model = RandomForestClassifier(random_state=12345, n_estimators=100)
RFC_model = model.fit(features_train, target_train).score(features_valid, target_valid)
predicted_valid = model.predict(features_valid)
print("cлучайный лес", RFC_model)

cлучайный лес 0.8585


Логистическая регрессия.

In [15]:
model = LogisticRegression(random_state=12345, solver='liblinear')
LR_model = model.fit(features_train, target_train).score(features_valid, target_valid)
predicted_valid = model.predict(features_valid)
print("логистическая регрессия", LR_model)

логистическая регрессия 0.7815


<b>Решающее дерево</b><br>
Точность попадания по классам 0.858<br>

<b>Случайный лес</b><br>
Точность попадания по классам 0.8585<br>

<b>Логистическая регрессия</b><br>
Точность попадания по классам 0.7815<br>

Построим матрицы ошибок для моделей

Случайный лес

In [16]:
model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 100)
model_RFC.fit(features_train, target_train)
RFC_prediction = model_RFC.predict(features_valid)
confusion_matrix(target_valid, RFC_prediction)

array([[1521,   61],
       [ 222,  196]])

In [17]:
print("Полнота" , recall_score(target_valid, RFC_prediction))
print("Точность", precision_score(target_valid, RFC_prediction))
print("F1-мера", f1_score(target_valid, RFC_prediction))

Полнота 0.4688995215311005
Точность 0.7626459143968871
F1-мера 0.5807407407407408


Оценим модель в целом, не привязываясь к конкретному порогу

In [18]:
RFC_prob = model_RFC.predict_proba(features_valid)[:, 1]
auc_roc_RFC = roc_auc_score(target_valid, RFC_prob)
print("AUC-ROC", auc_roc_RFC)

AUC-ROC 0.8424114893025


Дерево решений

In [19]:
model_DTC = DecisionTreeClassifier(random_state=12345)
model_DTC.fit(features_train, target_train)
DTC_prediction = model_DTC.predict(features_valid)
confusion_matrix(target_valid, DTC_prediction)

array([[1376,  206],
       [ 220,  198]])

In [20]:
print("Полнота" , recall_score(target_valid, DTC_prediction))
print("Точность", precision_score(target_valid, DTC_prediction))
print("F1-мера", f1_score(target_valid, DTC_prediction))

Полнота 0.47368421052631576
Точность 0.4900990099009901
F1-мера 0.4817518248175182


Низкое значние F1- меры свидетельствует о низком качестве модели, проблема в точности.

Оценим модель в целом, не привязываясь к конкретному порогу

In [21]:
DTC_prob = model_DTC.predict_proba(features_valid)[:, 1]
auc_roc_DTC = roc_auc_score(target_valid, DTC_prob)
print("AUC-ROC", auc_roc_DTC)

AUC-ROC 0.6717346463503893


Логистическая регрессия

In [22]:
model_LR = LogisticRegression(solver = 'liblinear')
model_LR.fit(features_train, target_train)
LR_prediction = model_LR.predict(features_valid)
confusion_matrix(target_valid, LR_prediction)

array([[1543,   39],
       [ 398,   20]])

In [23]:
print("Полнота" , recall_score(target_valid, LR_prediction))
print("Точность", precision_score(target_valid, LR_prediction))
print("F1-мера", f1_score(target_valid, LR_prediction))

Полнота 0.04784688995215311
Точность 0.3389830508474576
F1-мера 0.08385744234800838


Оценим модель в целом, не привязываясь к конкретному порогу

In [24]:
LR_prob = model_LR.predict_proba(features_valid)[:, 1]
auc_roc_LR = roc_auc_score(target_valid, LR_prob)
print("AUC-ROC", auc_roc_LR)

AUC-ROC 0.6727584246214894


<b>Вывод</b><br>
Ни одна модель не показал хороший результат, вероятно проблема в дисбалансе классов.
В нашей выборке отрицательных ответов около 80% , положитительных  около 20%.
Случайный лес выдает заметно больше качественных предсказаний, но также сильно склоняется к ложно позитивным предсказаниям. Точность и качество также низкое.
Матрица показала, что дерево решений склонно выдавать позитивные предсказания, очень высокое количество ложных позитивных предсказания.
Логистическая регрессия выдает большое количество негативных предсказаний — высокая точность модели объясняется высокой долей негативных ответов в валидационной выборке.

В данных сущестует дисбаланс классов. Возможно он влияет на качество изучаемых моделей.

Все модели имеют recall ниже, чем percision, из-за перекоса данных, что влияет на F-меру.

## Борьба с дисбалансом

Напишем функцию увеличения обьектов положительного класса

In [25]:
def up(features, target, repeat):

    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_up = pd.concat([features_zeros] + [features_ones] * repeat)
    target_up = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_up, target_up = shuffle(features_up, target_up, random_state=12345)
    
    return features_up, target_up

увеличим количество положительных ответов в 4 раза

In [26]:
features_up, target_up = up(features_train, target_train, 4)

In [27]:
features_up.shape

(9588, 11)

In [28]:
target_up.mean()

0.49895702962035876

Баланс классов примерно равен

In [29]:
model_DTC = DecisionTreeClassifier(random_state=123)
DTC_score = model_DTC.fit(features_train, target_train).score(features_valid, target_valid)
    
model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 100)
RFC_score = model_RFC.fit(features_train, target_train).score(features_valid, target_valid)
    
model_LgR = LogisticRegression(solver = 'liblinear')
LR_score = model_LR.fit(features_train, target_train).score(features_valid, target_valid)
print("Точность:" "дерево решений", DTC_score, "случайный лес ", RFC_score, "логистческая регрессия", LR_score)

Точность:дерево решений 0.7845 случайный лес  0.8585 логистческая регрессия 0.7815


In [30]:
model_DTC = DecisionTreeClassifier(random_state=123)
DTC_score = model_DTC.fit(features_up, target_up).score(features_valid, target_valid)
    
model_RFC = RandomForestClassifier(random_state=12345, n_estimators = 100)
RFC_score = model_RFC.fit(features_up, target_up).score(features_valid, target_valid)
    
model_LgR = LogisticRegression(solver = 'liblinear')
LR_score = model_LR.fit(features_up, target_up).score(features_valid, target_valid)
print("Точность:" "дерево решений", DTC_score, "случайный лес ", RFC_score, "логистческая регрессия", LR_score)

Точность:дерево решений 0.7725 случайный лес  0.849 логистческая регрессия 0.6605


Точность моделей изменились не сильно.

Проверим изменилось ли качество ответов

Дерево решений

In [31]:
model_DTC_up = DecisionTreeClassifier(random_state=12345)
model_DTC_up.fit(features_up, target_up)
DTC_prediction_up = model_DTC_up.predict(features_valid)

In [32]:
print("Полнота" , recall_score(target_valid, DTC_prediction_up))
print("Точность", precision_score(target_valid, DTC_prediction_up))
print("F1-мера", f1_score(target_valid, DTC_prediction_up))
print("AUC-ROC", roc_auc_score(target_valid, DTC_prediction_up))

Полнота 0.4473684210526316
Точность 0.4722222222222222
F1-мера 0.4594594594594595
AUC-ROC 0.6576285847361767


Случайный лес

In [33]:
model_RFC_up = RandomForestClassifier(random_state=12345, n_estimators = 100)
model_RFC_up.fit(features_up, target_up)
RFC_prediction_up = model_RFC_up.predict(features_valid)

In [34]:
print("Полнота" , recall_score(target_valid, RFC_prediction_up))
print("Точность", precision_score(target_valid, RFC_prediction_up))
print("F1-мера", f1_score(target_valid, RFC_prediction_up))
print("AUC-ROC", roc_auc_score(target_valid, RFC_prediction_up))

Полнота 0.5358851674641149
Точность 0.6746987951807228
F1-мера 0.5973333333333333
AUC-ROC 0.7338085761467225


Логистическая регрессия

In [35]:
model_LR_up = LogisticRegression(solver = 'liblinear')
model_LR_up.fit(features_up, target_up)
LR_prediction_up = model_LR_up.predict(features_valid)

In [36]:
print("Полнота" , recall_score(target_valid, LR_prediction_up))
print("Точность", precision_score(target_valid, LR_prediction_up))
print("F1-мера", f1_score(target_valid, LR_prediction_up))
print("AUC-ROC", roc_auc_score(target_valid, LR_prediction_up))

Полнота 0.6722488038277512
Точность 0.34143377885783716
F1-мера 0.45286059629331177
AUC-ROC 0.6648222527356202


Попробуем сделать объекты частого класса не такими частыми, проведем уменьшение выборки

In [37]:
def down(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_down = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_down = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_down, target_down = shuffle(features_down, target_down, random_state=12345)
    
    return features_down, target_down

features_down, target_down = down(features_train, target_train, 0.25)

Дерево решений

In [38]:
model_DTC_down = DecisionTreeClassifier(random_state=12345)
model_DTC_down.fit(features_down, target_down) 
DTC_predicted_valid = model_DTC_down.predict(features_valid)

In [39]:
print("Полнота" , recall_score(target_valid, DTC_predicted_valid))
print("Точность", precision_score(target_valid, DTC_predicted_valid))
print("F1-мера", f1_score(target_valid, DTC_predicted_valid))
print("AUC-ROC", roc_auc_score(target_valid, model_DTC_down.predict_proba(features_valid)[:, 1]))

Полнота 0.6961722488038278
Точность 0.37890625
F1-мера 0.49072512647554806
AUC-ROC 0.6973275909000175


Случайный лес

In [40]:
model_RFC_down = RandomForestClassifier(random_state=12345)
model_RFC_down.fit(features_down, target_down) 
predicted_valid = model_RFC_down.predict(features_valid)

In [41]:
print("Полнота" , recall_score(target_valid, predicted_valid))
print("Точность", precision_score(target_valid, predicted_valid))
print("F1-мера", f1_score(target_valid, predicted_valid))
print("AUC-ROC", roc_auc_score(target_valid, model_RFC_down.predict_proba(features_valid)[:, 1]))

Полнота 0.7344497607655502
Точность 0.4865293185419968
F1-мера 0.5853193517635844
AUC-ROC 0.8445119738203111


Увеличение выборки справилось лучше.

<b> Вывод </b><br>
Показатели всех моделей улучшились. 
Изначально в данных был дисбаланс -  отрицательны ответов около 80% , положитительных около 20%, из-за чего обученная на этих данных модель не проходила проверку на адекватность. Все модели не первоначальных данных характеризовались высокой степенью ошибок и низким качеством взвешенной величины (F1) — модели показывали низкие результаты точности и полноты.

С помощью upsampling, мы увеличили количество положительных ответов в 4 раза, тем самым получив баланс классов примерно равный - 0.49895702962035876.

На новых данных все модели показали результат выше, чем на несбалансированной выборке. Лучшие показатели у модели случайный лес:<br>
Полнота 0.5358851674641149<br>
Точность 0.6746987951807228<br>
F1-мера 0.5973333333333333<br>
AUC-ROC 0.7338085761467225<br>

Также мы рассмотрели объекты частого класса не такими частыми, провели уменьшение выборки.<br>
Увеличение выборки справилось лучше.

## Тестирование модели

Обучим финальную модель. Так как лучшие показатели у модели Случайные лес, то и будем работать с этой моделью.

In [42]:
model = RandomForestClassifier(n_estimators=100, 
                               max_depth=6, 
                               random_state=12345, 
                               min_samples_split=2, 
                               min_samples_leaf=2
                                  )
model.fit(features_up, target_up)

RandomForestClassifier(max_depth=6, min_samples_leaf=2, random_state=12345)

In [43]:
model_RFC_final = model.predict(features_test)
model_RFC_final_v = model.predict_proba(features_test)[:, 1]

In [44]:
print("Полнота" , recall_score(target_test, model_RFC_final))
print("Точность", precision_score(target_test, model_RFC_final))
print("F1-мера", f1_score(target_test, model_RFC_final))
print("AUC-ROC", roc_auc_score(target_test, model_RFC_final_v))

Полнота 0.7186761229314421
Точность 0.5049833887043189
F1-мера 0.5931707317073172
AUC-ROC 0.8495422526237835


Создадим константную модель, которая любому экземпляру, каждой строке прогнозирует класс «0», чтобы внутри Series передать список из нулей длиной в target_test

In [45]:
target_predict_c = pd.Series([0]*len(target_test))
target_predict_c.value_counts()

0    2000
dtype: int64

Сравним показатель точности у финальной модели и константной модели.

In [46]:
print('accuracy_score константой модели:', accuracy_score(target_valid, target_predict_c))
print('accuracy_score финальной модели:', accuracy_score(target_test, model_RFC_final))

accuracy_score константой модели: 0.791
accuracy_score финальной модели: 0.7915


Так же сравним AUC-ROC у финальной модели и константной модели.

In [ ]:
print('AUC-ROC константой модели:', roc_auc_score(target_valid, target_predict_c))
print('AUC-ROC финальной модели:', roc_auc_score(target_test, model_RFC_final_v))

<b>Выводы</b><br>

В первоначальные данных наблюдался дисбаланс классов. В нашей выборке отрицательных ответов около 80% , положитительных около 20%. Случайный лес выдавал заметно больше качественных предсказаний, но также сильно склонялся к ложно позитивным предсказаниям. Точность и качество также было низким. Матрица показала, что дерево решений склонно выдавать позитивные предсказания, очень высокое количество ложных позитивных предсказания. Логистическая регрессия выдавала большое количество негативных предсказаний — высокая точность модели объясняется высокой долей негативных ответов в валидационной выборке.<br>

Разобрали вариант борьбы с дисбалансом с помощью upsampling, мы увеличили количество положительных ответов в 4 раза, тем самым получив баланс классов примерно равный - 0.49895702962035876.<br>

На новых данных все модели показали результат выше, чем на несбалансированной выборке. Лучшие показатели у модели случайный лес:<br>
Полнота 0.5358851674641149<br>
Точность 0.6746987951807228<br>
F1-мера 0.5973333333333333<br>
AUC-ROC 0.7338085761467225<br>

На новых данных все модели показали результат выше, чем на несбалансированной выборке. Лучшие показатели были у модели случайного леса:<br>

Полнота 0.5358851674641149<br>
Точность 0.6746987951807228<br>
F1-мера 0.5973333333333333<br>
AUC-ROC 0.7338085761467225<br>

Финальная модель прошла проверку на адекватность в сравнении с константной моделью:<br>
accuracy_score константой модели: 0.791<br>
accuracy_score финальной модели: 0.7915<br>

AUC-ROC константой модели: 0.5<br>
AUC-ROC финальной модели: 0.7648548655240598<br>


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: задача исследована
    - [x]  Исследован баланс классов
    - [x]  Изучены модели без учёта дисбаланса
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 3: учтён дисбаланс
    - [x]  Применено несколько способов борьбы с дисбалансом
    - [x]  Написаны выводы по результатам исследования
- [x]  Выполнен шаг 4: проведено тестирование
- [x]  Удалось достичь *F1*-меры не менее 0.59
- [x]  Исследована метрика *AUC-ROC*